In [35]:
# データ分析に必要なライブラリを読み込む
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [47]:
files = glob.glob("workspace/train/*.csv")
data_list = []
for file in files:
  data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)

/tmp/ipykernel_59355/397933912.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
/tmp/ipykernel_59355/397933912.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


In [48]:
def data_pre(df):

  nonnull_list = []
  for col in df.columns:
    nonnull = df[col].count()
    if nonnull == 0:
      nonnull_list.append(col)
  df = df.drop(nonnull_list, axis=1)

  df["市区町村コード"].value_counts()

  df = df.drop("種類", axis=1)

  distance = {
    "30分?60分":45,
    "1H?1H30":75,
    "2H?":120,
    "1H30?2H":105
  }
  df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(distance).astype(float)

  df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上",2000).astype(float)

  y_list = {}
  for yearStr in df["建築年"].value_counts().keys():
    if "平成" in yearStr:
      num = float(yearStr.split("平成")[1].split("年")[0])
      year = 33 - num
    if "令和" in yearStr:
      num = float(yearStr.split("令和")[1].split("年")[0])
      year = 3 - num
    if "昭和" in yearStr:
      num = float(yearStr.split("昭和")[1].split("年")[0])
      year = 96 - num
      
    y_list[yearStr] = year
  y_list["戦前"] = 76
  df["建築年"] = df["建築年"].replace(y_list)

  year = {
    "年第１四半期":".25",
    "年第２四半期":".50",
    "年第３四半期":".75",
    "年第４四半期":".99"
  }
  yearList = {}
  for yearStr in df["取引時点"].value_counts().keys():
    for k, j in year.items():
      if k in yearStr:
        yearReg = yearStr.replace(k, j)
    
    yearList[yearStr] = yearReg
  df["取引時点"] = df["取引時点"].replace(yearList).astype(float)
  
  # 数値以外のデータ型を"category"と明示
  for col in ["都道府県名", "市区町村名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
    df[col] = df[col].astype("category")
    
  return df

In [49]:
df = data_pre(df)

In [50]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

# モデル学習用のデータ、学習後の検証用データに分割(2割が検証用)
df_train, df_val = train_test_split(df, test_size=0.2)

# 目的変数を設定し目的変数を取り除いた各従属変数を設定する
col = "取引価格（総額）_log"

# 学習用データ
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

# 検証用データ
val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

# light-bgmに渡すパラメータ
params = {
  "objective":"regression",
  "metrics": "mae"
}



model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)

/usr/local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062818 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12168
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 18
[LightGBM] [Info] Start training from score 7.217630


/usr/local/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l1: 0.252766
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.235549
[3]	valid_0's l1: 0.220439
[4]	valid_0's l1: 0.206902
[5]	valid_0's l1: 0.19486
[6]	valid_0's l1: 0.184202
[7]	valid_0's l1: 0.175069
[8]	valid_0's l1: 0.166989
[9]	valid_0's l1: 0.159428
[10]	valid_0's l1: 0.152777
[11]	valid_0's l1: 0.147003
[12]	valid_0's l1: 0.14175
[13]	valid_0's l1: 0.137193
[14]	valid_0's l1: 0.133106
[15]	valid_0's l1: 0.12935
[16]	valid_0's l1: 0.126031
[17]	valid_0's l1: 0.12294
[18]	valid_0's l1: 0.120242
[19]	valid_0's l1: 0.117862
[20]	valid_0's l1: 0.11572
[21]	valid_0's l1: 0.113582
[22]	valid_0's l1: 0.111813
[23]	valid_0's l1: 0.109871
[24]	valid_0's l1: 0.108245
[25]	valid_0's l1: 0.106812
[26]	valid_0's l1: 0.105407
[27]	valid_0's l1: 0.104251
[28]	valid_0's l1: 0.103067
[29]	valid_0's l1: 0.101972
[30]	valid_0's l1: 0.100944
[31]	valid_0's l1: 0.100069
[32]	valid_0's l1: 0.0991862
[33]	valid_0's l1: 0.0983675
[34]	valid_0's l1: 0.09755

In [51]:
# modelに検証用データを渡す
vals = model.predict(val_x)
mae(vals, val_y)

0.07652242696512324